+ 暴落後GUの数が急激に増えているように見える
1. 実際増えているのか？
1. FD率はどのくらいか

In [ ]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor

from quantopian.pipeline.experimental import QTradableStocksUS
from quantopian.pipeline.data.builtin import USEquityPricing

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

class ValueDayBeforeYesterday(CustomFactor):
    window_length = 2
    def compute(self, today, assets, out, values):
        out[:] = values[0]

class Gap(CustomFactor): 
    window_length = 2
    def compute(self, today, assets, out, values1, values2):
        out[:] = values2[-1] / values1[0]-1
        
def make_pipeline():
    pipe = Pipeline()
    base_universe = QTradableStocksUS()
    
    prev_close = ValueDayBeforeYesterday(inputs = [USEquityPricing.close])
    prev_high = ValueDayBeforeYesterday(inputs = [USEquityPricing.high])
    pipe.add(USEquityPricing.close.latest, 'close')
    pipe.add(prev_close, 'prev_close')
    pipe.add(prev_high, 'prev_high')
    pipe.add(USEquityPricing.open.latest, 'open')
    gap_up = Gap(inputs = [USEquityPricing.open, USEquityPricing.close])
    gapup_over5p = gap_up > 0.05
    pipe.add(gap_up, "gap_up")
    pipe.add(gapup_over5p, "gapup_over5p")
    
    pipe.set_screen(base_universe)
    return pipe



In [ ]:
start_date='2013-1-1'
end_date='2018-1-31'
pipeline_results = run_pipeline(make_pipeline(), start_date=start_date, end_date=end_date)
spy = get_pricing("spy", start_date=start_date, end_date=end_date,fields="price",frequency='daily')

In [ ]:
cnt = pipeline_results[pipeline_results["gapup_over5p"]].groupby(level=0).count()["gapup_over5p"]
df = pd.concat([spy, cnt], axis=1)
df["spy return"] = df[symbols("spy")].pct_change()

In [ ]:

df.plot(y ="gapup_over5p", x="spy return", kind='scatter')

In [ ]:
pd.concat([spy, cnt], axis=1).plot(secondary_y="gapup_over5p")

In [ ]:
df["2014/10/1":"2014/12/31"].plot(secondary_y="gapup_over5p")

In [ ]:
plt.scatter(x = df["spy return"].shift(1), y =df["gapup_over5p"] )

### ↑前日のSPYのリターンが大きい方が次の日GAPUP銘柄が増える？

In [ ]:
df["prev spy return"] = df["spy return"].shift(1)

In [ ]:
def get_gapup_filldown(pipeline_results, dates):
    """
    pipeline_resultsからdates のデータだけ抜き出し，
    さらに，gapup > 5% した銘柄だけを取得
    その銘柄の分足データをGetし，
    09：01とその後を比較．
    全データをDataFrame化して，転置し，コラムにt分，indexに銘柄名が来るようにする
    """
    df_targets = pipeline_results.ix[dates]
    l = list()
    for date in dates: 
        tickers = df_targets[df_targets["gapup_over5p"]].ix[date].index 
        x = get_pricing(tickers, start_date=date, end_date=date, fields='price', frequency='minute')
        y = x / x.iloc[0] - 1
        l.append(y.reset_index().drop("index", axis=1))
       
    dfT = pd.concat(l, axis=1).T 
    return dfT 


In [ ]:
dates = df[df["prev spy return"]<-0.02].index
dfT = get_gapup_filldown(pipeline_results, dates)
dfT.plot.scatter(x=10,y=40)

In [ ]:
## 
dates = df[df["prev spy return"]>0.02].index
dfT = get_gapup_filldown(pipeline_results, dates)
dfT.plot.scatter(x=10,y=40)


In [ ]:
dates = df["gapup_over5p"].sort_values(ascending=False)[:100].index
dfT = get_gapup_filldown(pipeline_results, dates)
dfT.plot.scatter(x=30,y=70)

+ ⇑から判断するに
+ GAPUP５％以上が160以上あった場合（：100はだいたい160以上だったので）
+ Open３０分で，６％以上上がった銘柄と５％下がった銘柄は，トレンド通りポジションを持って
+ Open７０分後にクローズ

+ 結果．勝てない.アルゴリズムはこちら https://www.quantopian.com/algorithms/5a780935c25e40000fbf4b9b
